# Model Checking Practical Session

Practical session of Automated Software Engineering (BMEVIMIAC20), 2024

by Levente Bajczi, ftsrg (Critical Systems Research Group), Budapest University of Technology and Economics

based on previous work by Csanád Telbisz

## Overview

The purpose of this session is to get familiar with the ways programs can be analyzed using automated, mathematically precise techniques. 
We will use the Z3 SMT solver to first solve some problems semi-manually, then we will take a look at ready-to-use automated program verifiers.

# Part 1: SMT Solving

SMT (Satisfiability Modulo Theories) is a problem that tries to find a satisfying assignment to variables such that some expression evaluates to _true_. For example, the `a*a + b*b = c*c` expressions evaluates to _true_ (is _satisfiable_ (**SAT**)) if `a := 3, b:= 4, c := 5` (but other _satisfying_ assignments exist), and `a <= 0 && a >= 1` has no such assignment, therefore, is _unsatisfiable_ (**UNSAT**). 

### Install required software

In [ ]:
!pip install z3-solver


### Required Imports

The required import for Z3 is:

In [ ]:
from z3 import *

There is a bunch of SMT solvers with different advantages and disadvantages. Some of them also have a Python API which are even interchangeable with the Z3 API in some cases (e.g., the [cvc5 Python API](https://cvc5.github.io/docs/cvc5-1.1.2/api/python/python.html)). You may try other solvers if you like, but we will stick to Z3 in this session as one of the most widely used and stable SMT solvers.

### Overview of the Python SMT API

#### Declaration of Various Kinds: Constants/Variables, Functions, etc.

One of the first steps when interfacing SMT solvers is to declare the **types** (sorts) and **named symbols** needed for modelling.

Using Python, we can declare **variables** (or constants, if you prefer) using the type name as a constructor. As a convention, the solver variable is usually assigned to a Python variable of the same name, which is later used to construct expressions. The available sorts include the ones introduced in the various SMT-LIB theories (integers, floating points, arrays, ...). 

Pay attention to the definition of integers. In the Integer SMT Theory, integers are _mathematical_, i.e., have no upper or lower bound. When we model integers from programming languages, we usually rely on _bitvector_ types, which are fix-width integers. Bit-vector variables are declared by adding the bit-width as a second argument.

In [ ]:
# variable x ranging over the reals
x = Real('x')

# variable z ranging over the integers
z = Int('z')

# boolean variable b
b = Bool('b')

# 32-bit bitvector bv
bv = BitVec('bv', 32)

#### Constructing Expressions

Terms and formulas can be built using "Pythonic" syntax by using mathematical/infix notation. The available operators correspond to those defined in SMT-LIB; for the details, we refer to the [section on theories](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-theories) in the _Programming Z3_ article.

For simple tasks, we can also use the function `solve` to check satisfiability of a formula, and compute a solution.

In [ ]:
# two variables over Reals
x = Real('x')
y = Real('y')

# terms consisting of expressions
term_equality = x == y + 1.5
term_nonzero  = y != 0
# a conjunction (logical 'and') of two equations
f = And(term_equality, term_nonzero)

# compute a solution of the formula
solve(f)

In [ ]:
# conjoining another inequality makes the formula UNSAT
solve(f, x < y)

**Exercise 1**

Invert the following real-valued matrix:
$$
A = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix}
$$


In [ ]:
M = [[1, 2],
     [3, 4]]

# declare four variables over the reals to represent the elements of the inverse matrix M^-1
# M^-1 = |a b|
#        |c d|

#TODO: your answer here

# formulate four equations characterising the inverse, use the function solve
# (hint: formulate the matrix product M * M^-1 = I manually for each element of I)

#TODO: your answer here


## Part 2: Program Verification 

Next, we will try to use SMT queries to answer a program verification task.

Our goal is to prove that in the following program, the assertion never fails:

``` c++

char i = input();
char j = i + 1;
assert(j > i);

```

Logically, the assertion should not fail, because we take an input number, add 1 to it, and assert that it became bigger. Right?

**Exercise 2**

  Encode the above program's statements as SMT terms!

In [ ]:
i = Int('i')
j = Int('j')

#TODO: your answer here

solve(j_assignment, assertion_fail)

Notice that there was no satisfying solution -- does this mean our program can never fail?

Copy the code below (which contains the exact same logic, just instrumented to compile properly) to a new file (`test.c`), compile and run it, and try to cause an assertion violation!

``` c++
int main() {
  char i;
  printf("Give me i: ");
  scanf("%hhd", &i);
  char j = i + 1;
  if(!(j > i)) printf("Assertion violation!\n");
}
```

For compilation, you can use `gcc -o test test.c`, after which type `./test` to execute your program.

(Hint: `char` is an 8-bit signed number. Try it around its boundary values! If you're unsure, consult [this](https://www.tutorialspoint.com/cprogramming/c_data_types.htm) resource.)

**Exercise 2b**

Re-encode the program's statements to use 8-bit bitvectors instead!

In [ ]:
i = BitVec('i', 8)
j = BitVec('j', 8)

#TODO: your answer here

solve(j_assignment, assertion_fail)

## Part 3: Program Verification in Practice


Next, we will use an automated program verification tool to answer a program verification task.

Our goal is to prove that in the following program, the function `reach_error()` is never called (`__VERIFIER_nondet{bool,int}` are _nondeterministic_ values, which models, among others, user input):

<details><summary>trex01-1.c</summary>
    
``` c++
void reach_error();

_Bool __VERIFIER_nondet_bool();
int __VERIFIER_nondet_int();

void f(int d) {
  int x = __VERIFIER_nondet_int(), y = __VERIFIER_nondet_int(), k = __VERIFIER_nondet_int(), z = 1;
  if (!(k <= 1073741823))
    return;
  L1:
  while (z < k) { z = 2 * z; }
  if(!(z>=2)) reach_error();
  L2:
  while (x > 0 && y > 0) {
    _Bool c = __VERIFIER_nondet_bool();
    if (c) {
      P1:
      x = x - d;
      y = __VERIFIER_nondet_bool();
      z = z - 1;
    } else {
      y = y - d;
    }
  }
}

int main() {
  _Bool c = __VERIFIER_nondet_bool();
  if (c) {
    f(1);
  } else {
    f(2);
  }

  return 0;
}
```

</details>

Even though this is not too big of a program, you probably still don't want to encode it by hand. So let's use a tool that can help in verifying this program.

#### fm-weck: Run formal methods tools in containerized environments

Verification tools are complex software, usually with many dependencies. Instead of trying to install them directly, we rely on a convenience package called `fm-weck`, which is a command line tool and library that makes the execution of formal methods tools easy. fm-weck downloads the tools and caches them for later use.

In [ ]:
!pip install fm-weck

After installing the command line tool, we can go ahead and install some software verification tools on our machine. For two examples, we will use [Ultimate Automizer](https://www.ultimate-pa.org/automizer/), and [CPAchecker](https://cpachecker.sosy-lab.org/). 

In [ ]:
!fm-weck install uautomizer:svcomp24 cpachecker:2.3.1

Next, we can run one of the tools on the input file `trex01-1.c`:

In [ ]:
!fm-weck run uautomizer:svcomp24 --property unreach-call trex01-1.c 2>/dev/null | tail

Check the line after `Result:`. If it is `FALSE`, the verification found an error, i.e., a path to the `reach_error()` function. Otherwise, if `TRUE` is there, it proved the program's safety.

**Exercise**: Check the contents of the `output` folder. Pay special attention to the `UltimateCounterExample.errorpath` file: what can you see? Can you see _why_ the output is what it is?


**Exercise**: Delete the `output` folder, and run the other tool (`cpachecker:2.3.1`) as well! What can you see in its output? Which is "better" (which gives more information on the result)?

**Exercise**: Try to modify the source code in the `trex01-1.c` file so that it changes the result with one of the tools! Be creative, but beware: if you make the problem much harder to solve, the tools might not finish in time!

# Further Exercises, References and Other Resources

If you are done with all tasks, we recommend that you head over to [Dennis Yurichev's excellent SAT/SMT by example](https://smt.st/) collection of puzzles and exercises!

This material was prepared based on the [practical session](https://sat-smt-ar-school.gitlab.io/www/2024/schedule.html) of Philipp Rümmer (University of Regensburg & Uppsala University) at the SAT/SMT/AR Summer School, 27 June 2024.

You can find detailed introductions to SMT-LIB and to the Python APIs, among others, here:

* [The SMT-LIB website](https://smt-lib.org)
* [David Cok's SMT-LIB Tutorial](https://smtlib.github.io/jSMTLIB/SMTLIBTutorial.pdf)
* [Dennis Yurichev's SAT/SMT by example](https://smt.st/)
* [Leonardo de Moura's Z3Py Tutorial](https://scungao.github.io/mit-iap17/z3python.html)
* [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html) by Nikolaj Bjørner, Leonardo de Moura, Lev Nachmanson, and Christoph Wintersteiger
* [Z3Py Namespace Reference](https://z3prover.github.io/api/html/namespacez3py.html)
